# Encoding .png stimuli images into VA-VQE Vectors

Vector Quantization and Variational Autoencoder 
Vector Quantization converts data into discrete blocks or codes 
Variational Autoencoder Compresses and Reconstructs data

In [10]:
!pip install accelerate
!pip install git+https://github.com/huggingface/diffusers

  Cloning https://github.com/huggingface/diffusers to /private/var/folders/jt/46g4wcz94mb72f5z3kq461s00000gn/T/pip-req-build-gurajbqz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /private/var/folders/jt/46g4wcz94mb72f5z3kq461s00000gn/T/pip-req-build-gurajbqz
  Resolved https://github.com/huggingface/diffusers to commit 00b179fb1afc147f87bd311f03b1ef7d747e1792
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3718795 sha256=78ffb949d297a01952e396cd7d35781c0e537661c1e9cf8ddc04036aab67adfd
  Stored in directory: /private/var/folders/jt/46g4wcz94mb72f5z3kq461s00000gn/T/pip-ephem-wheel-cache-navdgt0z/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [diffusers]/3 [diffusers]


In [1]:
import torch 
from torchvision import transforms
from PIL import Image
from diffusers.models import AutoencoderKL


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pytorch we will use for machine learning
torchvision lets us resize,normalize and covert images to tensors
PIL or Pillow is for handling .png files
Diffuser Models lets autoencode images by VQ-VAE proccess

In [2]:
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
vae.eval().to("mps")

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

This Auto encoder is used for vae compression and decompression

In [3]:
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


Here we transform the image to size 512x512
to pytorch tensor and normalize.

this is done for autoencoder as it expects 512x512 images

In [5]:
import os
IMAGE_DIR = '/Volumes/Samsung_PSSD_T7_Shield/NSD dataset/FULL DATASET/prepared_nsd_data'
Z_SAVE_DIR = os.path.join(IMAGE_DIR, 'z_latents')
os.makedirs(Z_SAVE_DIR, exist_ok=True)


In [6]:
all_images = sorted([
    f for f in os.listdir(IMAGE_DIR)
    if f.startswith('image_') and f.endswith('.png') and f[6:-4].isdigit()
])

In [7]:
print(f"Found {len(all_images)} well-formatted images.")


Found 195000 well-formatted images.


These are for finding the images in the folder

In [ ]:
from tqdm import tqdm
for fname in tqdm(all_images, desc="Extracting VQ-VAE features"):
        image_path = os.path.join(IMAGE_DIR, fname)
        img = Image.open(image_path).convert("RGB")
        img_tensor = preprocess(img).unsqueeze(0).to("mps")
        with torch.no_grad():
            z = vae.encode(img_tensor).latent_dist.sample() * 0.18215
            torch.save(z.cpu(), os.path.join(Z_SAVE_DIR, fname.replace('.png', '.pt')))


        




Extracting VQ-VAE features:   0%|       | 608/195000 [14:43<78:22:18,  1.45s/it]